In [1]:
# Predicción de Precios de Vehículos Usados con Regresión Lineal
# 
# Este proyecto implementa un modelo de regresión lineal para predecir precios de vehículos usados
# utilizando características como año, kilometraje, tipo de combustible, etc.

# Importación de librerías necesarias
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, median_absolute_error
import pickle
import zipfile
import gzip
import json
import os
import pandas as pd

In [2]:
# Paso 1: Carga de datos desde archivos comprimidos
# Los datos están divididos en conjuntos de entrenamiento y prueba

# Cargar datos de prueba
with zipfile.ZipFile('../files/input/test_data.csv.zip', 'r') as zip:
    with zip.open('test_data.csv') as f:
        df_test = pd.read_csv(f)

# Cargar datos de entrenamiento  
with zipfile.ZipFile('../files/input/train_data.csv.zip', 'r') as zip:
    with zip.open('train_data.csv') as f:
        df_train = pd.read_csv(f)

In [3]:
# Paso 2: Función para preprocesar los datos
def clean_data(df):
    """
    Preprocesa los datos del dataset:
    - Crea la columna 'Age' a partir del año (asumiendo año actual 2021)
    - Elimina columnas 'Year' y 'Car_Name'
    - Elimina filas con valores faltantes
    """
    df = df.copy()
    df['Age'] = 2021 - df['Year']
    df = df.drop(columns=['Year', 'Car_Name'])
    df = df.dropna()
    return df

In [4]:
# Aplicar limpieza a los datos
df_test_clean = clean_data(df_test)
df_train_clean = clean_data(df_train)

# Dividir los datasets en características (X) y variable objetivo (y)
# La variable objetivo es 'Present_Price'
x_train = df_train_clean.drop('Present_Price', axis=1)
y_train = df_train_clean['Present_Price']
x_test = df_test_clean.drop('Present_Price', axis=1) 
y_test = df_test_clean['Present_Price']

In [5]:
# Paso 3: Crear pipeline del modelo
def create_model_pipeline():
    """
    Crea un pipeline que incluye:
    - One-hot encoding para variables categóricas
    - Escalado MinMax para variables numéricas 
    - Selección de las K mejores características
    - Modelo de regresión lineal
    """
    # Definir tipos de variables
    categories = ["Fuel_Type", "Selling_type", "Transmission"]  
    numerics = ["Selling_Price", "Driven_kms", "Age", "Owner"]

    # Preprocessor para transformar las variables
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', OneHotEncoder(handle_unknown='ignore'), categories),
            ('scaler', MinMaxScaler(), numerics)
        ],
        remainder='passthrough'
    )

    # Selector de características
    selectkbest = SelectKBest(score_func=f_regression)

    # Pipeline completo
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('selectkbest', selectkbest),
        ('classifier', LinearRegression())
    ])

    return pipeline

In [6]:
# Paso 4: Función para optimizar hiperparámetros
def optimize_hyperparameters(model, n_splits, x_train, y_train, scoring):
    """
    Optimiza los hiperparámetros del modelo usando validación cruzada
    - Prueba diferentes valores de K para la selección de características
    - Usa validación cruzada con n_splits divisiones
    - Optimiza según la métrica de scoring especificada
    """
    estimator = GridSearchCV(
        estimator=model,
        param_grid={
            "selectkbest__k": range(1, 13),  # Probar K de 1 a 12
        },
        cv=n_splits,
        refit=True,
        scoring=scoring
    )
    estimator.fit(x_train, y_train)
    return estimator

In [7]:
# Crear y entrenar el modelo con optimización de hiperparámetros
model_pipeline = create_model_pipeline()

# Optimizar usando validación cruzada con 10 splits
# Usar error absoluto medio negativo como métrica de optimización
optimized_model = optimize_hyperparameters(
    model_pipeline, 
    n_splits=10, 
    x_train=x_train, 
    y_train=y_train, 
    scoring='neg_mean_absolute_error'
)

c:\Users\david\LAB-13-prediccion-de-precios-usando-regresion-lineal-DavidZapataN\.venv\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:783: UserWarning: k=12 is greater than n_features=11. All the features will be returned.
  warnings.warn(
c:\Users\david\LAB-13-prediccion-de-precios-usando-regresion-lineal-DavidZapataN\.venv\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:783: UserWarning: k=12 is greater than n_features=11. All the features will be returned.
  warnings.warn(
c:\Users\david\LAB-13-prediccion-de-precios-usando-regresion-lineal-DavidZapataN\.venv\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:783: UserWarning: k=12 is greater than n_features=11. All the features will be returned.
  warnings.warn(
c:\Users\david\LAB-13-prediccion-de-precios-usando-regresion-lineal-DavidZapataN\.venv\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:783: UserWarning: k=12 is greater than n_features=11. All 

In [8]:
# Paso 6: Función para calcular métricas de evaluación
def calculate_metrics(model, x_train, y_train, x_test, y_test):
    """
    Calcula métricas de evaluación para conjuntos de entrenamiento y prueba:
    - R² (coeficiente de determinación)
    - MSE (error cuadrático medio)
    - MAD (error absoluto mediano)
    """
    # Predicciones
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)

    # Métricas para conjunto de entrenamiento
    train_metrics = {
        'type': 'metrics',
        'dataset': 'train',
        'r2': r2_score(y_train, y_train_pred),
        'mse': mean_squared_error(y_train, y_train_pred),
        'mad': median_absolute_error(y_train, y_train_pred)
    }

    # Métricas para conjunto de prueba
    test_metrics = {
        'type': 'metrics',
        'dataset': 'test',
        'r2': r2_score(y_test, y_test_pred),
        'mse': mean_squared_error(y_test, y_test_pred),
        'mad': median_absolute_error(y_test, y_test_pred)
    }

    return train_metrics, test_metrics

In [9]:
# Calcular métricas de evaluación del modelo
train_metrics, test_metrics = calculate_metrics(
    optimized_model, 
    x_train, 
    y_train, 
    x_test, 
    y_test
)

In [10]:
# Paso 5: Funciones para guardar resultados
def save_model(model):
    """
    Guarda el modelo entrenado en formato comprimido
    """
    os.makedirs('../files/models', exist_ok=True)
    with gzip.open('../files/models/model.pkl.gz', 'wb') as f:
        pickle.dump(model, f)

def save_metrics(metrics):
    """
    Guarda las métricas en formato JSON
    Cada línea del archivo contiene un diccionario con las métricas
    """
    os.makedirs('../files/output', exist_ok=True)
    with open("../files/output/metrics.json", "w") as f:
        for metric in metrics:
            json_line = json.dumps(metric)
            f.write(json_line + "\n")

In [11]:
# Guardar el modelo entrenado y las métricas
save_model(optimized_model)
save_metrics([train_metrics, test_metrics])